In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 39.2904,
	"longitude": -76.6122,
	"start_date": "1960-03-16",
	"end_date": "1960-03-16",
	"hourly": ["temperature_2m", "precipitation", "cloud_cover", "wind_speed_100m"],
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)



Coordinates 39.26185989379883°N -76.611572265625°E
Elevation 29.0 m asl
Timezone b'America/New_York' b'EST'
Timezone difference to GMT+0 -18000 s
                        date  temperature_2m  precipitation  cloud_cover  \
0  1960-03-16 05:00:00+00:00           -7.52            0.0          0.0   
1  1960-03-16 06:00:00+00:00           -9.07            0.0          0.0   
2  1960-03-16 07:00:00+00:00          -10.37            0.0          0.0   
3  1960-03-16 08:00:00+00:00          -11.37            0.0         93.0   
4  1960-03-16 09:00:00+00:00          -11.62            0.0         65.0   
5  1960-03-16 10:00:00+00:00          -12.42            0.0         99.0   
6  1960-03-16 11:00:00+00:00          -11.77            0.0        100.0   
7  1960-03-16 12:00:00+00:00          -10.02            0.0        100.0   
8  1960-03-16 13:00:00+00:00           -6.92            0.0        100.0   
9  1960-03-16 14:00:00+00:00           -5.27            0.0        100.0   
10 1960-03-16 15:0